
# Subscription Tracker 



## Açıklama

Bu uygulama, aboneliklerini takip etmek isteyen kullanıcılar için tasarlandı.
 - Aylık / 3 Aylık / Yıllık abonelik desteği
 - “Ödendi” butonu aynı gün sadece 1 kez çalışır
 - Geçmiş ödemeler otomatik olarak ileri taşınır
 - Aylık toplam gider + yaklaşan ödemeler kartları

In [1]:
# Gerekli kütüphaneler
from flask import Flask, render_template_string, request, redirect, url_for, flash
import json, os, smtplib, ssl
from email.message import EmailMessage
from datetime import datetime, date
from calendar import monthrange
from typing import Optional

DATA_FILE = "subscriptions.json"  # Abonelik verilerinin saklanacağı JSON dosyası
APP_SECRET = os.getenv("APP_SECRET", "dev-secret")  # Flash mesajlar için gizli anahtar

# (Opsiyonel) E-posta bildirimleri için ayarlar
SMTP_HOST = os.getenv("SMTP_HOST", "")
SMTP_PORT = int(os.getenv("SMTP_PORT", "587"))
SMTP_USER = os.getenv("SMTP_USER", "")
SMTP_PASS = os.getenv("SMTP_PASS", "")
NOTIFY_TO = os.getenv("NOTIFY_TO", "")

# Flask uygulaması oluşturuluyor
app = Flask(__name__)
app.secret_key = APP_SECRET

In [2]:
# JSON tabanlı veri depolama


def load_data():
    """JSON dosyasını okuyup abonelik listesini döndürür."""
    if not os.path.exists(DATA_FILE): 
        return []
    with open(DATA_FILE, "r", encoding="utf-8") as f:
        return json.load(f)

def save_data(data):
    """Abonelik listesini JSON dosyasına kaydeder."""
    with open(DATA_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)


In [3]:
# Tarih işlemleri (güvenli ay ekleme, gün hesaplama)


def add_months(d: date, months: int = 1) -> date:
    """Bir tarihe ay ekler (31 gibi farklı uzunluklara uyum sağlar)."""
    y = d.year + (d.month - 1 + months) // 12
    m = (d.month - 1 + months) % 12 + 1
    last_day = monthrange(y, m)[1]
    return date(y, m, min(d.day, last_day))

def days_until(yyyymmdd: str) -> Optional[int]:
    """Belirtilen tarihe kaç gün kaldığını döndürür."""
    try:
        due = datetime.strptime(yyyymmdd, "%Y-%m-%d").date()
        return (due - date.today()).days
    except Exception:
        return None


In [4]:
# E-posta bildirimi (isteğe bağlı)

def can_send_email() -> bool:
    """SMTP bilgileri tam ise True döner."""
    return all([SMTP_HOST, SMTP_PORT, SMTP_USER, SMTP_PASS, NOTIFY_TO])

def send_email(subject: str, body: str):
    """E-posta gönderir (ayarlar doğruysa)."""
    if not can_send_email(): 
        return False
    try:
        msg = EmailMessage()
        msg["From"] = SMTP_USER
        msg["To"] = NOTIFY_TO
        msg["Subject"] = subject
        msg.set_content(body)

        context = ssl.create_default_context()
        with smtplib.SMTP(SMTP_HOST, SMTP_PORT) as server:
            server.starttls(context=context)
            server.login(SMTP_USER, SMTP_PASS)
            server.send_message(msg)
        return True
    except Exception:
        return False


In [5]:
TEMPLATE = """
<!doctype html>
<html lang="tr" data-bs-theme="dark">
<head>
  <meta charset="utf-8">
  <title>Abonelik Takip</title>
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css" rel="stylesheet">
  <style>
    body { background:#0b0e12; }
    .card { background:#131822; border:1px solid #1e2633; }
    .badge { font-size:.85rem }
  </style>
</head>
<body class="text-light">

<div class="container py-5">
  <h1 class="mb-4">📅 Subscription Tracker</h1>

  <!-- Bildirimler -->
  {% with messages = get_flashed_messages() %}
    {% if messages %}
      <div class="alert alert-info">{{ messages[-1] }}</div>
    {% endif %}
  {% endwith %}

  <!-- Özet Kartları -->
  <div class="row mb-4">
    <div class="col-md-6">
      <div class="card p-3">
        <h5>Aylık Toplam Maliyet</h5>
        <p class="h4 text-warning">{{ "%.2f"|format(monthly_total) }} ₺</p>
      </div>
    </div>
    <div class="col-md-6">
      <div class="card p-3">
        <h5>En Yakın Ödemeler</h5>
        {% if upcoming %}
          <ul class="list-unstyled m-0">
          {% for u in upcoming %}
            <li>📌 {{ u.name }} → {{ u.renewal_date }} ({{ u.days_left }} gün)</li>
          {% endfor %}
          </ul>
        {% else %}
          <p class="text-secondary">Yakında ödeme yok.</p>
        {% endif %}
      </div>
    </div>
  </div>

  <!-- Abonelik Ekleme Formu -->
  <div class="card p-3 mb-4">
    <form class="row g-3" method="post" action="{{ url_for('add_subscription') }}">
      <div class="col-md-3">
        <input class="form-control" name="name" placeholder="Abonelik Adı (Netflix)" required>
      </div>
      <div class="col-md-2">
        <input class="form-control" type="number" step="any" name="price" placeholder="Tutar (₺)" required>
      </div>
      <div class="col-md-3">
        <label class="form-label mb-1">Son Ödeme Tarihi</label>
        <input class="form-control" type="date" name="renewal_date" required>
      </div>
      <div class="col-md-2">
        <select class="form-select" name="cycle_months" required>
          <option value="1">Aylık</option>
          <option value="3">3 Aylık</option>
          <option value="12">Yıllık</option>
        </select>
      </div>
      <div class="col-md-2 d-flex align-items-end">
        <button class="btn btn-primary w-100">Ekle / Güncelle</button>
      </div>
    </form>
  </div>

  <!-- Abonelik Listesi -->
  <div class="row g-3">
    {% for s in subs %}
    <div class="col-md-6">
      <div class="card p-3 h-100">
        <div class="d-flex justify-content-between">
          <h5 class="m-0">{{ s.name }}</h5>
          {% if s.status == "Aktif" %}
            <span class="badge bg-success">Aktif</span>
          {% elif s.status == "Yaklaşıyor" %}
            <span class="badge bg-warning text-dark">Yaklaşıyor</span>
          {% elif s.status == "Bugün" %}
            <span class="badge bg-info text-dark">Bugün</span>
          {% else %}
            <span class="badge bg-danger">Süresi Doldu</span>
          {% endif %}
        </div>
        <div class="small text-secondary mt-2">Son Ödeme: {{ s.renewal_date }}</div>
        <div><strong>Tutar:</strong> {{ "%.2f"|format(s.price) }} ₺</div>
        <div><strong>Periyot:</strong> {{ s.cycle_months }} ay</div>
        <div><strong>Kalan:</strong> {{ s.days_left }} gün</div>
        <div class="d-flex gap-2 mt-3">
          <form method="post" action="{{ url_for('delete_subscription') }}">
            <input type="hidden" name="name" value="{{ s.name }}">
            <button class="btn btn-outline-danger btn-sm">Sil</button>
          </form>
          <form method="post" action="{{ url_for('mark_paid') }}">
            <input type="hidden" name="name" value="{{ s.name }}">
            <button class="btn btn-outline-primary btn-sm">Ödendi</button>
          </form>
        </div>
      </div>
    </div>
    {% endfor %}
  </div>
</div>

</body>
</html>
"""


In [6]:
 def evaluate_and_notify(subs: list):
    """
    Abonelikleri inceler:
    - Gün farkını hesaplar
    - Geçmiş ödemeleri otomatik ileri alır
    - Durum etiketlerini günceller
    """
    today = date.today()
    banners = []

    for s in subs:
        s.setdefault("last_paid", "")
        s.setdefault("cycle_months", 1)
        days = days_until(s.get("renewal_date", ""))
        s["days_left"] = days

        if days is None:
            s["status"] = "Bilinmiyor"
            continue

        if days > 3:
            s["status"] = "Aktif"
        elif 1 <= days <= 3:
            s["status"] = "Yaklaşıyor"
        elif days == 0:
            s["status"] = "Bugün"
        else:
            # geçmiş ödemeleri otomatik ileri al
            try:
                due = datetime.strptime(s["renewal_date"], "%Y-%m-%d").date()
                new_due = add_months(due, int(s["cycle_months"]))
                s["renewal_date"] = new_due.strftime("%Y-%m-%d")
                s["days_left"] = (new_due - today).days
                s["status"] = "Aktif"
                banners.append(f"⏭️ {s['name']} otomatik {s['renewal_date']} tarihine taşındı.")
            except Exception:
                s["status"] = "Bilinmiyor"

    return subs, banners


In [7]:
@app.route("/")
def index():
    """Ana sayfa: abonelikleri listeler ve özetleri gösterir."""
    subs = load_data()
    subs, banners = evaluate_and_notify(subs)
    save_data(subs)

    if banners: flash(banners[-1])

    monthly_total = sum(float(s["price"]) for s in subs if s.get("days_left") is not None)
    upcoming = sorted([s for s in subs if s.get("days_left") is not None and s["days_left"] >= 0],
                      key=lambda x: x["days_left"])[:5]

    return render_template_string(TEMPLATE, subs=subs,
                                  monthly_total=monthly_total,
                                  upcoming=upcoming)


In [8]:
@app.route("/add", methods=["POST"])
def add_subscription():
    """Yeni abonelik ekler veya mevcut olanı günceller."""
    subs = load_data()
    name = request.form["name"].strip()
    price = float(request.form["price"])
    renewal_date = request.form["renewal_date"]
    cycle_months = int(request.form.get("cycle_months", 1))

    # Geçmiş tarih kontrolü
    try:
        due = datetime.strptime(renewal_date, "%Y-%m-%d").date()
        if due < date.today():
            flash("❌ Son ödeme tarihi geçmiş olamaz. Lütfen ileri bir tarih girin.")
            return redirect(url_for("index"))
    except Exception:
        flash("❌ Geçersiz tarih formatı.")
        return redirect(url_for("index"))

    # Güncelle veya ekle
    found = False
    for s in subs:
        if s["name"].lower() == name.lower():
            s.update({"price": price, "renewal_date": renewal_date, "cycle_months": cycle_months})
            found = True
            break
    if not found:
        subs.append({"name": name, "price": price, "renewal_date": renewal_date,
                     "cycle_months": cycle_months, "last_paid": ""})

    save_data(subs)
    return redirect(url_for("index"))

@app.route("/delete", methods=["POST"])
def delete_subscription():
    """Aboneliği siler."""
    subs = load_data()
    name = request.form["name"].strip().lower()
    subs = [s for s in subs if s["name"].lower() != name]
    save_data(subs)
    return redirect(url_for("index"))

@app.route("/mark-paid", methods=["POST"])
def mark_paid():
    """Abonelik ödendiğinde, yeni döneme taşır."""
    subs = load_data()
    name = request.form["name"].strip().lower()
    today = date.today()

    for s in subs:
        if s["name"].lower() == name:
            last_paid = s.get("last_paid", "")
            # Aynı gün tekrar ödeme engeli
            if last_paid == today.strftime("%Y-%m-%d"):
                flash("⚠️ Bu abonelik bugün zaten ödendi olarak işaretlenmiş.")
                return redirect(url_for("index"))

            due = datetime.strptime(s["renewal_date"], "%Y-%m-%d").date()
            new_due = add_months(due, int(s.get("cycle_months", 1)))
            s["renewal_date"] = new_due.strftime("%Y-%m-%d")
            s["last_paid"] = today.strftime("%Y-%m-%d")
            flash(f"✅ {s['name']} ödendi. Yeni tarih: {s['renewal_date']}")
            break

    save_data(subs)
    return redirect(url_for("index"))


In [ ]:

# Uygulamayı Başlat

if __name__ == "__main__":
    app.run(debug=False, use_reloader=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Oct/2025 12:50:52] "GET / HTTP/1.1" 200 -
